In [ ]:
from openpyxl import load_workbook
import numpy as np
import copy
excel_document =load_workbook('Finalpadrinos.xlsx') #Nombre archivo, carpeta "usuario"
sheet = excel_document.active
matriz_ahijados=[]
matriz_padrinos=[]
lista_padrinos=[]
lista_ahijados=[]
multiple_cells = sheet['E2':'AC158']###Celdas en las que trabajaremos###
for row in multiple_cells:
    lista_auxiliar=[]
    for cell in row:
        lista_auxiliar.append(cell.value)
    if(lista_auxiliar[1]=="Padrino"): ###Importante: la segunda pregunta debe ser ahijado/padrino###
        matriz_padrinos.append(lista_auxiliar)
        lista_padrinos.append(lista_auxiliar[0])
    elif(lista_auxiliar[1]=="Ahijado"):
        matriz_ahijados.append(lista_auxiliar)
        lista_ahijados.append(lista_auxiliar[0])
##challa pura##
lista_padrinos.reverse()
lista_padrinos.append("vacio xdxd")
lista_padrinos.reverse()
###----------##
n_ahijados=len(matriz_ahijados)
n_padrinos=len(matriz_padrinos)
matriz_match=[]
matriz_match.append(lista_padrinos)
for ahijadito in matriz_ahijados:
    lista_puntajes=[]
    lista_puntajes.append(ahijadito[0])  ##Considerando que la posicion 0 es el nombre##
    for padrinito in matriz_padrinos:
        puntaje=0                   
        for i in range(2,25):  ##Consideramos 2 solo si son 2 preguntas previas tales como nombre y si es ahijado,numero total de preguntas+2
            if ahijadito[i]==padrinito[i]:
                puntaje+=1
        lista_puntajes.append(puntaje)
    matriz_match.append(lista_puntajes)
num=copy.deepcopy(matriz_match)
num.pop(0) #copia de matriz match sin nombres
for i in range(len(num)):
    num[i].pop(0)
for row in matriz_match:
    sheet.append(row)

padrinos=matriz_match[0]
ahijados=[]
for i in matriz_match:
    ahijados.append(i[0])
padrinos.pop(0)
ahijados.pop(0)

from docplex.mp.model import Model
import copy
mdl=Model('modelo')
#Parametros
I=len(num)
J=len(num[0])
ahijados_ = [i for i in range(I)]
padrinos_ = [j for j in range(J)]
#Diccionario
ahi_pad = [(i,j) for i in (ahijados_) for j in (padrinos_)]
pij= [num[i][j] for i in range(I) for j in range(J)]
Pij = dict(zip(ahi_pad,pij))
#Variable
x = mdl.binary_var_dict(ahi_pad, name = 'x')
#Funcion objetivo
mdl.maximize(mdl.sum(x[i]*Pij[i] for i in ahi_pad))
#restricciones
for i in ahijados_:
    mdl.add_constraint(mdl.sum(x[(i,j)] for j in padrinos_) == 1)

for j in padrinos_:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i in ahijados_) >= 1)
    mdl.add_constraint(mdl.sum(x[(i,j)] for i in ahijados_) <= 2)
    
print(mdl.export_to_string())
solucion=mdl.solve(log_output=True)
mdl.get_solve_status()

import pandas as pd
import numpy as np
f=solucion.get_value_df(x)
matriz_pos=[]
for i in range(len(f)):
    if(f.iloc[i][2]==1):
        x,y=f.iloc[i][0],f.iloc[i][1]
        matriz_pos.append([x.astype(np.int),y.astype(np.int)])
        
#lista_padrinos=matriz_match[0].remove('vacio xdxd')
lista_ahijados=[]
for i in range(len(matriz_match)):
    lista_ahijados.append(matriz_match[i][0])
parejas=[]
for i in matriz_pos:
    ahijado=lista_ahijados[i[0]]
    print(i[1])
    padrino=lista_padrinos[i[1]]
    parejas.append([ahijado,padrino])
for row in parejas:
    sheet.append(row)
#excel_document.save('Finalpadrinos.xlsx')